In [3]:
!pip install pandas

In [5]:
import pandas as pd

In [ ]:
import pandas as pd


data_path = 'http://localhost:8888/edit/Downloads/domain_sentiment_data.tar.gz' 
data = pd.read_csv(data_path)


print(data.head())

In [ ]:
import string


def clean_text(text):
    text = text.lower()                             
    text = text.translate(str.maketrans('', '', string.punctuation))  
    return text

data['cleaned_review'] = data['review'].apply(clean_text)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['cleaned_review'])
sequences = tokenizer.texts_to_sequences(data['cleaned_review'])


max_length = max([len(x) for x in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')


label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['sentiment'])  

In [ ]:

data = data[data['cleaned_review'].str.split().str.len() > 2]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(padded_sequences, data['label'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
import numpy as np
from keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))
    
    def __getitem__(self, index):
        batch_x = self.X[index*self.batch_size:(index+1)*self.batch_size]
        batch_y = self.y[index*self.batch_size:(index+1)*self.batch_size]
        return batch_x, batch_y

train_generator = DataGenerator(X_train, y_train, batch_size=32)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout


model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_length))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))  

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator, validation_data=(X_val, y_val), epochs=10)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
#simple web interface 

In [ ]:
!pip install Flask

In [ ]:
from flask import Flask, request, jsonify
from keras.models import load_model